<h1>CNN for to detect brain tumors using MRI images</h1>

<h3>Этап 1</h3> Разделение данных

In [1]:
import os
import shutil
from sklearn.model_selection import train_test_split

In [2]:
#определяем директории
DATA_DIR = "data/raw"
TRAIN_DIR = "data/train"
VAL_DIR = "data/val"
TEST_DIR = "data/test"
categories = ["yes", "no"]

In [3]:
#создаем директории
for dir_path in [TRAIN_DIR, VAL_DIR, TEST_DIR]:
    os.makedirs(dir_path, exist_ok=True)
    for category in categories:
        os.makedirs(os.path.join(dir_path, category), exist_ok=True)

In [7]:
#распределяем данные по директориям
def split_and_move_data(data_dir, train_dir, val_dir, test_dir, categories):
    for category in categories:
        category_path = os.path.join(data_dir, category)
        images = os.listdir(category_path)
        
        # Разделение на train, val, test
        train_images, temp_images = train_test_split(images, test_size=0.3, random_state=42)
        val_images, test_images = train_test_split(temp_images, test_size=0.5, random_state=42)
        
        # Перемещение файлов
        for image_set, target_dir in [(train_images, train_dir), 
                                      (val_images, val_dir), 
                                      (test_images, test_dir)]:
            for image in image_set:
                shutil.move(
                    os.path.join(category_path, image), 
                    os.path.join(target_dir, category, image)
                )

In [5]:
split_and_move_data(DATA_DIR, TRAIN_DIR, VAL_DIR, TEST_DIR, categories)
print("Данные успешно разделены и перемещены!")


Данные успешно разделены и перемещены!


<h3>Этап 2</h3>  Предобработка данных